# import Library

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

from PIL import Image
from collections import OrderedDict #for multiGPU
from tqdm import tqdm

import os
import random
import time

# GPU

In [2]:
# device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
#for colab
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device is ", device)

device is  cuda


# Load Data

In [3]:
print("loading dataset")

data_transforms = {
    'train' : transforms.Compose([ #horizontal flip, random crop, normalization           
            #transforms.ToPILImage(),
            # transforms.Resize((256,256)),
            transforms.RandomCrop(32, padding = 4), #augmentation, input for ResNet
            transforms.RandomHorizontalFlip(), #augmentation
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
            #transforms.Normalize((0.4914, 0.4822, 0.4465), (1, 1, 1)), #with the per-pixel mean subtracted

        ]) , 
        'test': transforms.Compose([
            transforms.ToTensor(), #single view of the original 32x32 image
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
        ])
    }
#access data_transforms['train']

#dataloader (batch)
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform = data_transforms['train']
)
trainloader = torch.utils.data.DataLoader(
    train_set, batch_size = 128, shuffle = True, num_workers=2
)

test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)

#classes
classes = ('plane', 'car', 'bird', 'cat', 'deer','dog', 'frog', 'horse', 'ship', 'truck')


loading dataset


100%|██████████| 170498071/170498071 [00:01<00:00, 102857450.01it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Model Architecture

In [4]:
class optionA(nn.Module):
  def __init__(self, func):
    super(optionA, self).__init__()
    self.func = func
  
  def forward(self, x):
    return self.func(x)

In [5]:
class Fx2_16(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(16, 16, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(16)
    )
    self.residual = nn.Sequential()
  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out +=self.residual(x)
    out = F.relu(out)

    return out

In [6]:
class Fx2_32(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32),
    )
    self.residual = nn.Sequential()
  
  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out += self.residual(x)
    out = F.relu(out)
    return out


In [7]:
class Fx2_64(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Sequential(
         nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    self.conv2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    self.residual = nn.Sequential()

  def forward(self, x):
    out = F.relu(self.conv1(x))
    out = self.conv2(out)
    out += self.residual(x)
    out = F.relu(out)
    return out

In [8]:
#classes for Resnet56 Model
class conv16 (nn.Module): #Blue
  def __init__(self):
    super().__init__()
    self.Fx1 = Fx2_16()
    self.Fx2 = Fx2_16()
    self.Fx3 = Fx2_16()
    self.Fx4 = Fx2_16()
    self.Fx5 = Fx2_16()
    self.Fx6 = Fx2_16()
    self.Fx7 = Fx2_16()
    self.Fx8 = Fx2_16()
    self.Fx9 = Fx2_16()
    
  def forward(self,x):
    x1 = self.Fx1(x)
    x2 = self.Fx2(x1)
    x3 = self.Fx3(x2) 
    x4 = self.Fx4(x3) 
    x5 = self.Fx5(x4) 
    x6 = self.Fx6(x5) 
    x7 = self.Fx7(x6) 
    x8 = self.Fx8(x7) 
    x9 = self.Fx9(x8)
    #x = self.relu(x)
    return x9

#classes for Resnet56 Model
class conv32 (nn.Module): #Purple
  def __init__(self):
    super().__init__()
    self.Fx1_1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(16, 32, kernel_size = 3, stride = 2, padding = 1, bias = False),
        nn.BatchNorm2d(32))
    self.Fx1_2 = nn.Sequential(
        nn.Conv2d(32, 32, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(32)
    )

    ## OptionB
    # self.residual_mapping = nn.Sequential(
    #     nn.Conv2d(16,32,kernel_size =1, stride = 2, bias= False),
    #     nn.BatchNorm2d(32)
    # )

    # ## OptionA
    self.residual_mapping = optionA(lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, 32//4, 32//4), "constant", 0))

    self.Fx2 = Fx2_32()
    self.Fx3 = Fx2_32()
    self.Fx4 = Fx2_32()
    self.Fx5 = Fx2_32()
    self.Fx6 = Fx2_32()
    self.Fx7 = Fx2_32()
    self.Fx8 = Fx2_32()
    self.Fx9 = Fx2_32()
    # self.relu = nn.ReLU()
    
  def forward(self,x):
    out = F.relu(self.Fx1_1(x))
    out = self.Fx1_2(out) 
    out += self.residual_mapping(x)
    x1 = F.relu(out)
    
    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2) 
    x4 = self.Fx4(x3) 
    x5 = self.Fx5(x4) 
    x6 = self.Fx6(x5) 
    x7 = self.Fx7(x6) 
    x8 = self.Fx8(x7) 
    x9 = self.Fx9(x8) 
    # x2 = self.Fx2(x1) + x1
    # x3 = self.Fx3(x2) + x2
    #x = self.relu(x)
    return x9

#classes for Resnet56 Model
class conv64 (nn.Module): #Green
  def __init__(self):
    super().__init__()
    self.Fx1_1 = nn.Sequential(
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)
        nn.Conv2d(32, 64, kernel_size = 3, stride = 2, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )
    self.Fx1_2 = nn.Sequential(
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
        nn.BatchNorm2d(64)
    )

    ## OptionB
    # self.residual_mapping = nn.Sequential(
    #     nn.Conv2d(32,64,kernel_size =1, stride = 2, bias= False),
    #     nn.BatchNorm2d(64)
    # )

    ## OptionA
    self.residual_mapping = optionA(lambda x: F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, 64//4, 64//4), "constant", 0))
    
    self.Fx2 = Fx2_64()
    self.Fx3 = Fx2_64()
    self.Fx4 = Fx2_64()
    self.Fx5 = Fx2_64()
    self.Fx6 = Fx2_64()
    self.Fx7 = Fx2_64()
    self.Fx8 = Fx2_64()
    self.Fx9 = Fx2_64()

    # self.relu = nn.ReLU()
    
  def forward(self,x):
    out = F.relu(self.Fx1_1(x))
    out = self.Fx1_2(out)
    out += self.residual_mapping(x)
    x1 = F.relu(out)
    x2 = self.Fx2(x1) 
    x3 = self.Fx3(x2) 
    x4 = self.Fx4(x3) 
    x5 = self.Fx5(x4)
    x6 = self.Fx6(x5) 
    x7 = self.Fx7(x6) 
    x8 = self.Fx8(x7) 
    x9 = self.Fx9(x8) 

    return x9
   


In [9]:
class MyModel(nn.Module): #Resnet56 architecture implementation
    def __init__(self, conv2 = conv16, conv3 = conv32,
            conv4 = conv64, num_classes = 10, init_weights = True): #num_classes
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size = 3, stride = 1, padding=1, bias = False),
            nn.BatchNorm2d(16)
        )
        self.conv2 = conv2()
        self.conv3 = conv3()
        self.conv4 = conv4()
        #self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fclayer = nn.Linear(64, num_classes)

        if init_weights:
            self.initialize_weights()
    
    # define weight initialization function
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0) 
        
    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = self.conv2(x1)
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x = F.avg_pool2d(x4, x4.size()[3])
        #x = self.avg_pool(x4)
        x = x.view(x.size(0), -1)
        output = self.fclayer(x)
        return output

In [ ]:
resnet56 = MyModel()

In [ ]:
from torchsummary import summary
summary(resnet56, (3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 32, 32]           2,304
       BatchNorm2d-4           [-1, 16, 32, 32]              32
            Conv2d-5           [-1, 16, 32, 32]           2,304
       BatchNorm2d-6           [-1, 16, 32, 32]              32
            Fx2_16-7           [-1, 16, 32, 32]               0
            Conv2d-8           [-1, 16, 32, 32]           2,304
       BatchNorm2d-9           [-1, 16, 32, 32]              32
           Conv2d-10           [-1, 16, 32, 32]           2,304
      BatchNorm2d-11           [-1, 16, 32, 32]              32
           Fx2_16-12           [-1, 16, 32, 32]               0
           Conv2d-13           [-1, 16, 32, 32]           2,304
      BatchNorm2d-14           [-1, 16,

In [ ]:
MyModel()

MyModel(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv2): conv16(
    (Fx1): Fx2_16(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (residual): Sequential()
    )
    (Fx2): Fx2_16(
      (conv1): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv2): Sequential(
        (0)

# Train

In [10]:
# #get_model when loading model parameters trained w/ multi-gpu
def get_model(model_name, checkpoint_path):
    print("start loading model parameter")
    model = model_name()
    state_dict = torch.load(checkpoint_path)

    new_state_dict = OrderedDict()

    for k, v in state_dict.items():
        name = k.replace('module.', '')
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    print("model parameter loaded")
    return model

In [11]:

def train(n_epochs, train_loader) :
 
    epochs = n_epochs
    train_dataloader = train_loader
    #get current time -> folder name
    train_date = time.strftime("%y%m%d_%H%M", time.localtime(time.time()))
    # print(train_date)

    #log path for tensorboard
    log_path = os.path.join(os.getcwd(),"logs",train_date)
    print("log_path is {}".format(log_path))

    #model path to save '.pth'file
    model_path = os.path.join(os.getcwd(),"models",train_date)
    print("model_path is {}".format(model_path))
    if os.path.isdir(model_path) == False:
            os.makedirs(model_path)

    #writer for tensorboard
    writer = SummaryWriter(log_path)
 
    # whether available GPU
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    #define model (처음부터 학습시)
    print("처음부터 학습")
    model = MyModel()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    ####### 이전 학습한거에 추가적으로 학습시
    # print("이전에 하던거 부터 학습")
    # model_name = MyModel
    # # /content/models/230412_2145/0.pth
    # checkpoint_path = './12.pth'  #수정 필요
    # model = get_model(model_name,checkpoint_path)
    # model.to(device)

    #######

    #define loss funcion, optimizer, (scheduler)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(),lr=0.1, momentum=0.9,weight_decay=0.0001)
    #lr = 0.025, momentum = 0.9, weight_decay = 0.0005(original)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [81,122], gamma = 0.1)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones = [81,122], gamma = 0.1)
    # 0.1 -> 0.01 (32K iterations = 81 epochs) -> 0.001 (48K iterations = 122 epochs)

    print("start training")
    #Train
    for epoch in range(epochs):

        model.train(True)
        len_trainbatch = len(train_dataloader)

        i = 0
        for inputs, targets in tqdm(train_dataloader):
            i = i+1
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

            writer.add_scalar('train/loss', loss.item(), epoch*len_trainbatch+i) #iterations
        
        scheduler.step()    
        print("train loss of {}th epoch is {}".format(epoch,loss.item())) ## 수정 필요
 
        # Save every N(30) epoch
        save_epoch = 30
        if save_epoch > 0 and epoch % save_epoch == save_epoch-1:
            state_dict = model.state_dict()
            torch.save(state_dict, os.path.join(model_path, str(epoch+1)+'.pth')) #epoch+1로 바꾸기
        
    print('Finished Training')

    #save model after training
    torch.save(model.state_dict(), './model.pth') 

   

# Main

In [13]:

def main() :

    random.seed(53)
    #train(163, trainloader) #64k iterations = 163 epochs
    train(163, trainloader) #64k iterations = 163 epochs

   

if __name__ == '__main__':
    main()

log_path is /content/logs/230417_0053
model_path is /content/models/230417_0053
처음부터 학습
start training


100%|██████████| 391/391 [00:37<00:00, 10.44it/s]


train loss of 0th epoch is 2.030055522918701


100%|██████████| 391/391 [00:38<00:00, 10.18it/s]


train loss of 1th epoch is 1.5957404375076294


...

train loss of 160th epoch is 0.006305375602096319


100%|██████████| 391/391 [00:38<00:00, 10.07it/s]


train loss of 161th epoch is 0.002765841316431761


100%|██████████| 391/391 [00:38<00:00, 10.05it/s]

train loss of 162th epoch is 0.017061877995729446
Finished Training


# Test

- model

100 100 Loss: 0.344 | Acc: 92.530% (9253/10000)


- 30

100 Loss: 0.549 | Acc: 82.450% (8245/10000)


- 60

100 100 Loss: 0.475 | Acc: 85.630% (8563/10000)

- 90

100 100 Loss: 0.285 | Acc: 92.240% (9224/10000)


- 120

100 100 Loss: 0.343 | Acc: 92.430% (9243/10000)

- 150

100 100 Loss: 0.341 | Acc: 92.460% (9246/10000)

In [24]:
def test():

    ####### not touch
    model_name = MyModel
    # /content/models/230412_2145/0.pth
    #/content/models/230412_2208/1.pth
    #/content/models/230416_0841/90.pth
    #/content/models/230417_0053/30.pth
    checkpoint_path = './models/230417_0053/150.pth'  #수정 필요
    mode = 'test' 
#     data_dir = "./test_data"
#     meta_path = "./answer.json"
    model = get_model(model_name,checkpoint_path)
    #######
    
#     # Create training and validation datasets
#     test_datasets = MyDataset(meta_path, data_dir, data_transforms['test'],is_train= False) #self,meta_path,root_dir,transform=None, is_train = True

#     # Create training and validation dataloaders
#     test_dataloader = torch.utils.data.DataLoader(test_datasets, batch_size=batch_size, shuffle=False, num_workers=4)

    test_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=data_transforms['test'])
    
    testloader = torch.utils.data.DataLoader(
    test_set, batch_size=100, shuffle=False, num_workers=2)
    
    # Detect if we have a GPU available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"
    
    # Send the model to GPU
    #multi-GPU
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model = model.to(device)

    # Set model as evaluation mode
    for param in model.parameters():
        param.requires_grad = False
    model.eval()
    
    # Inference
    test_loss = 0
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    with torch.no_grad():
        batch_idx = 0
        for inputs, targets in tqdm(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            batch_idx = batch_idx + 1
            print(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))



In [25]:

def main() :

    random.seed(53)
    test()

   
if __name__ == '__main__':
    main()

start loading model parameter
model parameter loaded
Files already downloaded and verified


  2%|▏         | 2/100 [00:00<00:17,  5.47it/s]

1 100 Loss: 0.119 | Acc: 95.000% (95/100)
2 100 Loss: 0.205 | Acc: 93.500% (187/200)


  6%|▌         | 6/100 [00:00<00:07, 12.48it/s]

3 100 Loss: 0.181 | Acc: 94.667% (284/300)
4 100 Loss: 0.230 | Acc: 93.750% (375/400)
5 100 Loss: 0.257 | Acc: 93.800% (469/500)
6 100 Loss: 0.247 | Acc: 94.167% (565/600)


...

85 100 Loss: 0.341 | Acc: 92.376% (7852/8500)
86 100 Loss: 0.344 | Acc: 92.349% (7942/8600)
87 100 Loss: 0.343 | Acc: 92.356% (8035/8700)
88 100 Loss: 0.342 | Acc: 92.398% (8131/8800)
89 100 Loss: 0.341 | Acc: 92.382% (8222/8900)
90 100 Loss: 0.340 | Acc: 92.422% (8318/9000)


 96%|█████████▌| 96/100 [00:03<00:00, 28.65it/s]

91 100 Loss: 0.339 | Acc: 92.451% (8413/9100)
92 100 Loss: 0.338 | Acc: 92.467% (8507/9200)
93 100 Loss: 0.341 | Acc: 92.398% (8593/9300)
94 100 Loss: 0.342 | Acc: 92.372% (8683/9400)
95 100 Loss: 0.341 | Acc: 92.389% (8777/9500)
96 100 Loss: 0.342 | Acc: 92.375% (8868/9600)


100%|██████████| 100/100 [00:04<00:00, 24.30it/s]

97 100 Loss: 0.340 | Acc: 92.423% (8965/9700)
98 100 Loss: 0.340 | Acc: 92.439% (9059/9800)
99 100 Loss: 0.340 | Acc: 92.444% (9152/9900)
100 100 Loss: 0.341 | Acc: 92.460% (9246/10000)
